# Data Preparation
find number of suitable book

In [2]:
import json
import pandas as pd

# Load the goodreads_book_works.json file (JSONL format - one JSON object per line)
data = []
with open('goodreads_book_works.json', 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():
            data.append(json.loads(line))

# Convert to DataFrame for easier analysis
df = pd.DataFrame(data)

# Display the shape and columns
print(f"Total records: {len(df)}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())

Total records: 1521962
Columns: ['books_count', 'reviews_count', 'original_publication_month', 'default_description_language_code', 'text_reviews_count', 'best_book_id', 'original_publication_year', 'original_title', 'rating_dist', 'default_chaptering_book_id', 'original_publication_day', 'original_language_id', 'ratings_count', 'media_type', 'ratings_sum', 'work_id']

First few rows:
  books_count reviews_count original_publication_month  \
0           1             6                          8   
1          22         10162                              
2           2           268                              
3          38         89252                          7   
4           2            49                              

  default_description_language_code text_reviews_count best_book_id  \
0                                                    1      5333265   
1                                                  741        25717   
2                                                 

In [8]:
# Convert text_reviews_count to numeric
df['text_reviews_count'] = pd.to_numeric(df['text_reviews_count'], errors='coerce')

# Find books with text_reviews_count close to 1250
# Define "close to" as within a range (e.g., 1200-1300)
target = 1250
tolerance = 50

# Filter books close to 1250
close_to_target = df[(df['text_reviews_count'] >= target - tolerance) & 
                     (df['text_reviews_count'] <= target + tolerance)]

print(f"Books with text_reviews_count between {target - tolerance} and {target + tolerance}:")
print(f"Total found: {len(close_to_target)}\n")

# Display best_book_id, original_title, rating_dist, and text_reviews_count
result = close_to_target[['best_book_id', 'original_title', 'rating_dist', 'text_reviews_count']].sort_values('text_reviews_count')
print(result.to_string(index=False))


Books with text_reviews_count between 1200 and 1300:
Total found: 626

best_book_id                                                                                                                                       original_title                                        rating_dist  text_reviews_count
     7856305                                                                                                                                           Cross Fire       5:8318|4:9882|3:5258|2:733|1:160|total:24351                1200
      128092                                                                                                                                           The Pigman      5:5024|4:7581|3:7178|2:2470|1:886|total:23139                1200
     7789885                                                                                                                                   La fille de papier        5:3680|4:3343|3:1782|2:499|1:215|total:9519                12

In [11]:
# Get top 20 books closest to 1250 text reviews
# Calculate distance from target
close_to_target['distance_from_target'] = abs(close_to_target['text_reviews_count'] - target)

# Sort by distance and get top 20
top_20 = close_to_target.nsmallest(20, 'distance_from_target')[['best_book_id', 'original_title', 'text_reviews_count', 'rating_dist', 'distance_from_target']].reset_index(drop=True)

print("Top 20 books closest to 1250 text reviews:\n")
for idx, row in top_20.iterrows():
    print(f"{idx + 1}. Book ID: {row['best_book_id']}")
    print(f"   Title: {row['original_title']}")
    print(f"   Text Reviews Count: {row['text_reviews_count']}")
    print(f"   Rating Distribution: {row['rating_dist']}")
    print(f"   Distance from 1250: {row['distance_from_target']}")
    print()


Top 20 books closest to 1250 text reviews:

1. Book ID: 15724338
   Title: Any Duchess Will Do
   Text Reviews Count: 1250
   Rating Distribution: 5:4346|4:4949|3:2349|2:436|1:131|total:12211
   Distance from 1250: 0

2. Book ID: 17571140
   Title: Four Seconds to Lose
   Text Reviews Count: 1250
   Rating Distribution: 5:6328|4:5848|3:2180|2:385|1:86|total:14827
   Distance from 1250: 0

3. Book ID: 17558070
   Title: 
   Text Reviews Count: 1250
   Rating Distribution: 5:5578|4:4299|3:2211|2:739|1:355|total:13182
   Distance from 1250: 0

4. Book ID: 6674372
   Title: The Murderer's Daughters
   Text Reviews Count: 1250
   Rating Distribution: 5:2475|4:4887|3:4114|2:915|1:180|total:12571
   Distance from 1250: 0

5. Book ID: 6327989
   Title: Forge
   Text Reviews Count: 1250
   Rating Distribution: 5:3965|4:4240|3:1659|2:332|1:113|total:10309
   Distance from 1250: 0

6. Book ID: 12568505
   Title: Black City (Black City, #1)
   Text Reviews Count: 1250
   Rating Distribution: 5:334

C:\Users\kenpo\AppData\Local\Temp\ipykernel_19284\330764710.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_to_target['distance_from_target'] = abs(close_to_target['text_reviews_count'] - target)
